<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [ ]:
%load_ext tensorboard

In [ ]:
!rm -rf ./logs/ 

In [ ]:
import tensorflow as tf
import datetime
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data.head()

,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


In [1]:
#Import the relevant libraries
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn
from sklearn.model_selection import train_test_split
from keras.callbacks import Callback,ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
#import keras.backend as K
from sklearn.metrics import roc_auc_score
from tensorflow.keras import backend as K
from tensorflow.keras import metrics

In [ ]:
#Splitted the train and the test data with train size=75% and test size=25%
train_data,test_data = train_test_split(data, test_size=0.25, random_state=100)

In [ ]:
print(train_data.shape)
print(test_data.shape)

(15000, 3)
(5000, 3)


In [ ]:
#Getting the independent and dependent variable for train dataset
y_train = train_data.pop('label')
x_train =train_data

In [ ]:
#Getting the independent and dependent variable for test dataset
y_test = test_data.pop('label')
x_test = test_data

In [ ]:
y_train.value_counts()

0.0    7577
1.0    7423
Name: label, dtype: int64

In [ ]:
#Describe the callbacks on train begin which will initialize all the metrics list for each epoch and then on_epoch_end all the metrics values are getting appending. Here I have taken the
#help from Callback_Reference.ipynb file provided by AAIC to describe the below two methods.
#We will call the LossHistory class later while using the callback
class LossHistory(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': [],'f1':[],'auc':[], 'val_f1':[],'val_auc':[]}

    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('accuracy'))
        if logs.get('val_loss', -1) != -1:
             self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
             self.history['val_acc'].append(logs.get('val_accuracy'))
        self.history['f1'].append(logs.get('f1'))
        self.history['auc'].append(logs.get('auc'))
        if logs.get('val_f1', -1) != -1:
             self.history['val_f1'].append(logs.get('val_f1'))
        if logs.get('val_auc', -1) != -1:
             self.history['val_auc'].append(logs.get('val_auc'))    
        

In [ ]:
#Define a common model architechture for the first model and the second models as in both the model the architechture will remain same apart from the activation function. We are using
#sequential model here to define the architechture. In the upcoming assignment we will be using the functional architechture. However the snippet is taken as a reference from the
#Callback.reference of AAIC
def create_model(activation_name):
  return tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation= activation_name),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(8, activation= activation_name),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation= activation_name),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation= activation_name),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation= activation_name),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation= "sigmoid")
  ])

  

In [ ]:
#Now we will define the f1 score function and auc function below. Here to define the f1 function we are taking the help of backend.Keras class and in order to define the auc we are taking
#the help of py_function from tensorflow
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def auc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

def changeLearningRate(epoch):
      initial_learningrate=0.1
      changed = initial_learningrate*(1-0.1)**epoch
      print('The learning rate is:')
      print(changed)
      return changed

class LearningRateScheduler:
  def scheduler(epoch, lr):
    lr=0.0001
    if epoch < 10:
      return lr*50
    else:
      return lr * tf.math.exp(-0.1)

In [ ]:
model = create_model('tanh')

#history_own = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=True)
myhistory = LossHistory()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[f1,auc,'accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#lrschedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=True)
model.fit(x=x_train, 
          y=y_train, 
          epochs=2, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback,myhistory])

logs/fit/20201029-115342
Epoch 1/2

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

  1/469 [..............................] - ETA: 0s - loss: 0.8197 - f1: 0.2222 - auc: 0.2333 - accuracy: 0.3438WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
469/469 [==============================] - 2s 4ms/step - loss: 0.7055 - f1: 0.4677 -

{'acc': [0.5041333436965942, 0.5037333369255066],
 'auc': [0.5006610751152039, 0.5133196711540222],
 'f1': [0.457034170627594, 0.43319448828697205],
 'loss': [0.6959877014160156, 0.6932462453842163],
 'val_acc': [0.5188000202178955, 0.5852000117301941],
 'val_auc': [0.5260816812515259, 0.6258983016014099],
 'val_f1': [0.5589699745178223, 0.5504204630851746],
 'val_loss': [0.692583441734314, 0.6920908093452454]}